In [ ]:
# 내가 따라적은 코드

import pygame
import os   # pc에 저장된 디렉터리 구조에 접근하여 현재 경로 혹은 특정 경로에 파일의 유무 검사 가능

pygame.init()   # pygame 모듈 초기화

SCREEN_WIDTH = 1000
SCREEN_HEIGHT = 600

WHITE = (255, 255, 255)
BLACK = (0, 0, 0)
BLUE = (0, 0, 255)
RED = (255, 0, 0)
GREEN = (0, 255, 0)

# 화면 설정
# size = (SCREEN_WIDTH, SCREEN_HEIGHT)
# screen = pygame.display.set_mode(size)
screen = pygame.display.set_mode((SCREEN_WIDTH, SCREEN_HEIGHT))   # 위의 주석이 괄호 2개인 이유를 설명해줌.
pygame.display.set_caption("To do list")
clock = pygame.time.Clock()   # 프레임 초기 설정
font = pygame.font.SysFont("malgungothic", 48)   # 폰트 설정

FILE_NAME = "todolist.txt"    # 파일명 상수

class TaskManager:    # 할 일 목록을 관리하는 class
  def __init__(self):
    self.tasks = self.load_tasks()

  def load_tasks(self):
    if not os.path.exists(FILE_NAME):   # 모듈/ 모듈 안에 exists라는 함수기 존재.
      return {}   # 이는 딕셔너리 혹은 set.
    with open(FILE_NAME, 'r') as file:    # 읽기 전용 모드로 파일 오픈하여 작업 / context(현재 작업) 유지
      tasks = {}
      for line in file:
        task_id, task_desc = line.strip().split(":", 1)   # 딕셔너리에 1: ~~~ / 2: ~~~ / 3: ~~~ 과 같은 형태로 저장됨.
        tasks[int(task_id)] = task_desc
      return tasks

    def save_tasks(self):
      with open(FILE_NAME, 'w') as file:    # 쓰기 모드로 파일을 엶. 이는 파일이 없으면 알아서 생성을 해줌 / 파일이 있다면 기본적으로 덮어쓰기가 실행이 된다.
        for task_id, task_desc in self.tasks.items():   # items 함수는 딕셔너리 반복문에서 key랑 value를 동시에 return 해줌.
          file.write(f"{task_id}:{task_desc}\n")    # 딕셔너리 형식으로 출력해줌.

    def add_task(self, task_desc):
      task_id = max(self.tasks.key(), default = 0) + 1    # 새로운 딕셔너리 key를 생성
      self.tasks[task_id] = task_desc   # 새 key로 딕셔너리 추가
      self.save_tasks()

    def delete_task(self, task_id):
      if task_id in self.tasks:   # 딕셔너리에서 value가 key에 존재하는지 검사
        del self.tasks[task_id]
        self.save_tasks()

    def edit_task(self, task_id, new_desc):
      if task_id in self.tasks:
        self.tasks[task_id] = new_desc
        self.save_tasks()

    def get_task(self, task_id):
      return self.tasks.get(task_id)    # self.tasks[task_id] / task id에 접근하여 value만 출력

# 복잡한 문자열 입력을 처리하는 class
class TaskInputBox(pygame.Rect):    # Rect class 상속
  def __init__(self, x, y, w, h):
    super().__init__(x, y, w, h)    # 상속받은 class에서 기본적으로 있는 instance 변수들
    self.text = ''                  # 입력받을 문자열
    self.active = False             # 현재 입력받는 칸이 활성화 상태인지를 알려주는 변수

  def handle_event(self, event):
    if event.type == pygame.MOUSEBUTTONDOWN:    # 마우스 오른쪽 버튼 눌렀을 때
      if self.collidepoint(event.pos):    # 그 커서 위치가 현재 instance 위치와 겹친다면(위치 충돌)
        self.active = True
      else:
        self.active = False

    if event.type == pygame.KEYDOWN and self.active:    # 활성화 상태에서 키보드 입력이 일어났을 때
      if event.key == pygame.K_RETURN:    # Enter 키
        return self.text
      elif event.key == pygame.K_BACKSPACE:
        self.text = self.text[:-1]
      else:
        self.text += event.unicode    # unicode 값은 pc가 인식하는 문자의 값임. 문자로 변환해주는 역할
    return None

  def draw(self):
    color = BLUE
    pygame.draw.rect(screen, color, self, 2)    # 2 => 선의 굵기
    draw_text(self.text, position = (self.x + 5, self.y + 5))

def draw_text(strr, position = (50, 10), color = BLACK):    # mainloop 함수가 덜 지저분하게 하기 위한 함수
  text = font.render(strr, True, color)   # 글씨체 객체
  screen.blit(text, position)   # 화면 표시

def draw_tasks(tasks):
  nxt_y = 100
  for task_id, task_desc in tasks.items():
    draw_text(f"{task_id}. {task_desc}", position = (50, nxt_y))
    nxt_y += 50

def mainloop():
  task_manager = TaskManager()
  input_box = TaskInputBox(50, 100, 700, 70)
  edit_input_box = TaskInputBox(50, 100, 700, 70)
  edit_task_id = None

  mode = 'view'

  while True:
    screen.fill(WHITE)

    # mode에 따라 다른 화면 표시
    if mode == 'view':
      draw_text("Press 'a' to add, 'd' to delete, 'e' to edit")
      draw_tasks(task_manager.tasks)
    elif mode == 'add':   # 할 일을 추가하는 화면
      draw_text("Enter new task :")
      input_box.draw()
    elif mode == 'delete':    # id로 삭제하는 화면
      draw_text("Enter ID to delete")
      input_box.draw()
    elif mode == 'edit':    # id로 수정하는 화면
      # pass    # tab으로 구분된 제어문 / 함수 아래에서 아무것도 없어도 에러나는 것을 방지.
      if edit_task_id is None:
        draw_text("Enter ID to edit : ")
        input_box.draw()
      else:
        draw_text(f"Editing task {edit_task_id}. Enter new description.")
        edit_input_box.draw()


    # 키보드 이벤트 받기
    for event in pygame.event.get():
      if event.type == pygame.QUIT:
        pygame.quit()

      if mode == 'view':
        if event.type == pygame.KEYDOWN:    # 이벤트 종류
          if event.key == pygame.K_a:   # 눌린 키보드 키의 종류
            mode = 'add'
            input_box.text = ''
          elif event.key == pygame.K_d:
            mode = 'delete'
            input_box.text = ''
          elif event.key == pygame.K_e:
            mode = 'edit'
      elif mode == 'add':
        task_desc = input_box.handle_event(event)
        if task_desc is not None:
          task_manager.add_task(task_desc)
          mode = 'view'
      elif mode == 'delete':
        task_id = input_box.handle_event(event)
        if task_id is not None:
          try:
            task_id = int(task_id)    # task_id에 문자값이 입력되었다면 => 에러 발생
            task_manager.delete_task(task_id)
          except ValueError:
            pass    # error가 나도 무시하고 프로그램 실행 유지 가능
          mode = 'view'
      elif mode == 'edit':    # key 값을 숫자로 입력받으면 해당하는 value 값을 출력해주고, 수정할 수 있게 해주는 조건문
        if edit_task_id is None:
          task_id = input_box.handle_event(event)   # task_id 를 입력받음
          if task_id is not None:
            try:
              task_id = int(task_id)
              task_desc = task_manager.get_task(task_id)
              if task_desc is not None:
                edit_task_id = task_id
                edit_input_box.text = task_desc
            except ValueError:
              pass
        else:
          new_desc = edit_input_box.handle_event(event)
          if new_desc is not None:
            task_manager.edit_task(edit_task_id, new_desc)
            edit_task_id = None
            mode = 'view'

    pygame.display.flip()   # 화면을 업데이트해야 추가된 요소들이 비로소 보일 수 있음.
    # pygame.display.update()
    clock.tick(30)    # frame 수 설정

In [ ]:
# 강사님 코드

import pygame
import os

pygame.init()  # pygame 모듈 초기화

SCREEN_WIDTH = 1000
SCREEN_HEIGHT = 600

WHITE = (255, 255, 255)
BLACK = (0, 0, 0)
BLUE = (0, 0, 255)
RED = (255, 0, 0)
GREEN = (0, 255, 0)

# 화면 설정
#size = (SCREEN_WIDTH, SCREEN_HEIGHT)
#screen = pygame.display.set_mode(size)
screen = pygame.display.set_mode((SCREEN_WIDTH, SCREEN_HEIGHT))
pygame.display.set_caption('To do list')
clock = pygame.time.Clock()  # 프레임 초기 설정
font = pygame.font.SysFont("malgungothic", 48) # 폰트 설정

FILE_NAME = "todolist.txt"  # 파일명 상수

class TaskManager:
    def __init__(self):
        self.tasks = self.load_tasks()

    def load_tasks(self):
        if not os.path.exists(FILE_NAME):
            return {}
        with open(FILE_NAME, 'r') as file:  # context(현재 작업) 유지
            tasks = {}
            for line in file:
                task_id, task_desc = line.strip().split(':', 1)
                tasks[int(task_id)] = task_desc
            return tasks

    def save_tasks(self):
        with open(FILE_NAME, 'w') as file:
            for task_id, task_desc in self.tasks.items():
                file.write(f"{task_id}:{task_desc}\n")

    def add_task(self, task_desc):
        task_id = max(self.tasks.keys(), default=0) + 1 # 새로운 딕셔너리 key를 생성
        self.tasks[task_id] = task_desc  # 새 key로 딕셔너리에 추가
        self.save_tasks()

    def delete_task(self, task_id):
        if task_id in self.tasks:  # 딕셔너리에서는 key가 존재하는지 검사
            del self.tasks[task_id]
            self.save_tasks()

    def edit_task(self, task_id, new_desc):
        if task_id in self.tasks:
            self.tasks[task_id] = new_desc
            self.save_tasks()

    def get_task(self, task_id):
        return self.tasks.get(task_id)  # self.tasks[task_id], value만 리턴

# 복잡한 문자열 입력을 처리하는 class
class TaskInputBox(pygame.Rect): # Rect class 상속
    def __init__(self, x, y, w, h):
        super().__init__(x, y, w, h) # 상속받은 class에서 기본적으로 있는 instance 변수들
        self.text = ''               # 입력받을 문자열
        self.active = False          # 현재 입력받는 칸이 활성화 상태인지를 알려주는 변수

    def handle_event(self, event):
        if event.type == pygame.MOUSEBUTTONDOWN: # 마우스 오른쪽 버튼 눌렀을 때
            if self.collidepoint(event.pos): # 그 커서 위치가 현재 instance 위치와 겹친다면(위치 충돌)
                self.active = True
            else:
                self.active = False

        if event.type == pygame.KEYDOWN and self.active: # 활성화 상태에서 키보드 입력이 일어났을 때
            if event.key == pygame.K_RETURN: # Enter 키
                return self.text
            elif event.key == pygame.K_BACKSPACE:
                self.text = self.text[:-1]
            else:
                self.text += event.unicode
        return None

    def draw(self):
        color = BLUE
        pygame.draw.rect(screen, color, self, 2)
        draw_text(self.text, position=(self.x + 5, self.y + 5))

def draw_text(strr, position=(50, 10), color=BLACK):
    text = font.render(strr, True, color)  # 글씨체 객체
    screen.blit(text, position)  # 화면 표시

def draw_tasks(tasks):
    nxt_y = 100
    for task_id, task_desc in tasks.items():
        draw_text(f"{task_id}. {task_desc}", position = (50, nxt_y))
        nxt_y += 50

def mainloop():
    task_manager = TaskManager()
    input_box = TaskInputBox(50, 100, 700, 70)
    edit_input_box = TaskInputBox(50, 100, 700, 70)
    edit_task_id = None

    mode = 'view'

    while True:
        screen.fill(WHITE)

        # mode에 따라 다른 화면 표시
        if mode == 'view':
            draw_text("Press 'a' to add, 'd' to delete, 'e' to edit")
            draw_tasks(task_manager.tasks)
        elif mode == 'add':  # 할 일을 추가하는 화면
            draw_text("Enter new task : ")
            input_box.draw()
        elif mode == 'delete': # id로 삭제하는 화면
            draw_text("Enter ID to delete")
            input_box.draw()
        elif mode == 'edit': # id로 수정하는 화면
            # pass # tab으로 구분된 제어문/함수 아래서 아무것도 없어도 에러나는 걸 방지
            if edit_task_id is None:
                draw_text("Enter ID to edit : ")
                input_box.draw()
            else:
                draw_text(f"Editing task {edit_task_id}. Enter new description.")
                edit_input_box.draw()


        # 키보드 이벤트 받기
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()

            if mode == 'view':
                if event.type == pygame.KEYDOWN:  # 이벤트 종류
                    if event.key == pygame.K_a:   # 눌린 키보드 키의 종류
                        mode = 'add'
                        input_box.text = ''
                    elif event.key == pygame.K_d:
                        mode = 'delete'
                        input_box.text = ''
                    elif event.key == pygame.K_e:
                        mode = 'edit'
            elif mode == 'add':
                task_desc = input_box.handle_event(event)
                if task_desc is not None:
                    task_manager.add_task(task_desc)
                    mode = 'view'
            elif mode == 'delete':
                task_id = input_box.handle_event(event)
                if task_id is not None:
                    try:
                        task_id = int(task_id)
                        task_manager.delete_task(task_id)
                    except ValueError:
                        pass   # error가 나도 무시하고 프로그램 실행 유지 가능
                    mode = 'view'
            elif mode == 'edit':
                if edit_task_id is None:
                    task_id = input_box.handle_event(event)
                    if task_id is not None:
                        try:
                            task_id = int(task_id)
                            task_desc = task_manager.get_task(task_id)
                            if task_desc is not None:
                                edit_task_id = task_id
                                edit_input_box.text = task_desc
                        except ValueError:
                            pass
                else:
                    new_desc = edit_input_box.handle_event(event)
                    if new_desc is not None:
                        task_manager.edit_task(edit_task_id, new_desc)
                        edit_task_id = None
                        mode = 'view'

        pygame.display.flip()  # 화면을 업데이트해야 추가된 요소들이 비로소 보일 수 있음
        # pygame.display.update()
        clock.tick(30)   # frame 수 설정


if __name__ == "__main__":
    mainloop()